In [81]:
import argparse
from pathlib import Path
import sys
from datetime import datetime

wd = Path('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/workflows/BB-NF_17m')
sys.path.append('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/python')
from eddyproconfigeditor import EddyproConfigEditor

In [82]:
# arcc_wd = Path('/project/eddycovworkflow/afox18/Platinum-EddyPro7/workflows/BB-NF_17m')
arcc_wd = wd
# data_dir = Path('/gscratch/afox18/eddycovworkflow/InputData/Chimney')
data_dir = Path('/Users/alex/Documents/Data/Platinum_EC/InputData/Chimney')
out_dir = Path('/gscratch/afox18/eddycovworkflow/ExpectedOutputs/Chimney/BB-NF/17m')
out_dir = Path('/Users/alex/Documents/Data/Platinum_EC/ExpectedOutputs/Chimney/17m')

In [83]:
template = EddyproConfigEditor(wd / 'ini/BB-NF_17m_template.eddypro')

In [84]:
# start, end = '2019-01-01 00:00', '2023-12-31 23:30'
start, end = '2022-12-01 00:00', '2023-01-31 23:30'

In [85]:
template.Proj.set_metadata(arcc_wd / 'ini/BB-NF_17m.metadata')
template.Proj.set_biomet(mode='dir', path=data_dir / 'biomet/BB-NF/EC/17m/EddyPro_Biomet', extension='dat', subfolders=False)
template.Proj.set_project_name('BB-NF_17m')

In [86]:
template.Basic.set_raw_data(path=data_dir / 'raw/BB-NF/EC/17m/Calibrated', fmt='yyyy_mm_dd_HHMM.dat', subfolders=False)
template.Basic.set_out_path(out_dir / 'Template')
template.Basic.set_project_date_range(start=start, end=end)
template.Basic.set_missing_samples_allowance(pct=10)
template.Basic.set_flux_averaging_interval(minutes=30)
template.Basic.set_north_reference(method='mag')
template.Basic.set_output_id(output_id='template')

In [87]:
template.Adv.Proc.set_wind_speed_measurement_offsets(0, 0, 0)
template.Adv.Proc.set_axis_rotations_for_tilt_correction(
    method='planar_fit',
    configure_planar_fit_settings_kwargs=dict(
        w_max=0.5,
        u_min=0.5,
        num_per_sector_min=30,
        start=start,
        end=end,
        fix_method='CW',
        north_offset=0,
        sectors=[(False, 90)]*4
    )
)
template.Adv.Proc.set_turbulent_fluctuations(detrend_method='block')
template.Adv.Proc.set_timelag_compensations(method='covariance_maximization_with_default')
template.Adv.Proc.set_compensation_of_density_fluctuations(
    burba_method='multiple',
    day_bot=[3.0935, -0.0819, 0.0018, -0.3601],
    day_top=[0.5773, -0.0107, 0.0012, -0.0914],
    day_spar=[0.7714, -0.0154, 0.0011, -0.1164],
    night_bot=[2.2022, -0.122, 0, -0.3001],
    night_top=[-0.2505, -0.0303, 0, 0.0556],
    night_spar=[0.0219, -0.0361, 0, 0.0145]
)

In [88]:
template.Adv.Out.set_spectral_output(
    binned_spectra=True,
    binned_ogives=True,
    ensemble_spectra=True,
    full_spectra='none',
    full_cospectra='none'
)
template.Adv.Out.set_chain_of_custody(
    unprocessed='stats',
    despiked='stats',
    timelag_corrected='timeseries',
    variables=['u', 'v', 'w', 'ts', 'co2', 'h2o', ]
)

In [89]:
# worker_windows = [datetime(y, 1, 1, 0, 0, 0) for y in range(2019, 2025)]
worker_windows = [datetime(2022, 12, 1, 0, 0), datetime(2022, 12, 31, 0, 0), datetime(2023, 1, 31, 0, 0)]

In [90]:
template.to_eddypro(ini_file=wd / 'ini/BB-NF_17m_template.eddypro')
template.to_eddypro_parallel(
    ini_dir=wd / 'ini/BB-NF_17m_template_parallel',
    out_path=out_dir / 'Template_Parallel',
    file_duration=1440,
    worker_windows=worker_windows
)

In [92]:
test = EddyproConfigEditor('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/workflows/BB-NF_17m/test.ini').to_pandas()
temp = EddyproConfigEditor('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/workflows/BB-NF_17m/ini/BB-NF_17m_template.eddypro').to_pandas()

In [107]:
test.loc[test.Option == 'out_full_cosp_w_o'], temp.loc[temp.Option == 'out_full_cosp_w_o']

(                 Section             Option Value                Name
 329  RawProcess_Settings  out_full_cosp_w_o     0  BB-NF_17m_template,
 Empty DataFrame
 Columns: [Section, Option, Value, Name]
 Index: [])

In [105]:
set(test.Option).difference(temp.Option)

{'out_full_cosp_w_o'}